In [1]:
import pandas as pd
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

In [2]:
def draw_plates(df):
    df = (
        df.assign(row=lambda x: x.well_position.str[0:1])
        .assign(col=lambda x: x.well_position.str[1:])
    )
    wells = df[['row', 'col', 'control_type', 'pert_type']].copy()
    wells['col'] = wells.col.astype('int')
    wells['control_type'] = wells.control_type.fillna(wells['pert_type'])
    wells['cat'], uniques = pd.factorize(wells.control_type, sort=True)
    wells_pivot = wells.pivot('row', 'col', 'cat')

    sns.set(rc={'figure.figsize': (24, 16)})
    sns.set(font_scale=3)

    n_cat = wells.control_type.nunique()
    cat = list(uniques)
    color = 'tab10'
    colors = sns.color_palette(color, n_cat+1)[::-1]
    colors.pop(0)

    if n_cat != len(list(wells.control_type.drop_duplicates())):
        n_cat += 1
        cat.insert(0, 'empty')
        colors = sns.color_palette(color, n_cat)[::-1]

    cmap = LinearSegmentedColormap.from_list('Custom', colors, len(colors))

    ax = sns.heatmap(wells_pivot, cmap=cmap, linewidths=.5, linecolor='lightgray', square=True)

    colorbar = ax.collections[0].colorbar

    r = colorbar.vmax - colorbar.vmin
    colorbar.set_ticks([colorbar.vmin + 0.5 * r / (n_cat) + r * i / (n_cat) for i in range(n_cat)])
    colorbar.set_ticklabels(cat)

    ax.xaxis.tick_top()
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_xlabel('')

In [3]:
# Read plate map files
compound_platemap = pd.read_csv('../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_compound_platemap.txt', sep='\t')
crispr_platemap = pd.read_csv('../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_crispr_platemap.txt', sep='\t')
orf_platemap = pd.read_csv('../metadata/platemaps/2020_11_04_CPJUMP1/platemap/JUMP-Target-1_orf_platemap.txt', sep='\t')

In [4]:
# Read Metadata files
compound_metadata = pd.read_csv('../metadata/moa/JUMP-Target-1_compound_metadata.tsv', sep='\t')
crispr_metadata = pd.read_csv('../metadata/moa/JUMP-Target-1_crispr_metadata.tsv', sep='\t')
orf_metadata = pd.read_csv('../metadata/moa/JUMP-Target-1_orf_metadata.tsv', sep='\t')

In [5]:
# Plot compound plate map
compound_platemap_metadata = compound_platemap.merge(compound_metadata, on='broad_sample', how='left')
draw_plates(compound_platemap_metadata)
plt.savefig('figures/compound_platemap.png')
plt.close()

In [6]:
# Plot CRISPR plate map
crispr_platemap_metadata = crispr_platemap.merge(crispr_metadata, on='broad_sample', how='left')
draw_plates(crispr_platemap_metadata)
plt.savefig('figures/crispr_platemap.png')
plt.close()


In [7]:
# Plot ORF plate map
orf_platemap_metadata = orf_platemap.merge(orf_metadata, on='broad_sample', how='left')
draw_plates(orf_platemap_metadata)
plt.savefig('figures/orf_platemap.png')
plt.close()